## **1. 🩺 Introduction**

The Post-Operative Care Assistant is an AI-powered healthcare application designed to assist in monitoring and evaluating patients’ post-surgery recovery. By analyzing patient data and generating automated evaluation summaries, the system helps medical professionals make informed decisions and provide timely care.

## **2. Import Dependencies**

In [ ]:
# Core Python libraries
import os
import re
import json
import hashlib
import datetime
from pathlib import Path

# Data and computation
import numpy as np
import pandas as pd
from tqdm import tqdm

# AI/ML
import torch
from sentence_transformers import SentenceTransformer
from transformers import pipeline

# Vector storage
import faiss

# PDF processing
import PyPDF2

# Interface
import gradio as gr
from IPython.display import display

## **3. Load and Prepare Dataset**
Read or load medical/EHR text data, preprocess it, and chunk for embeddings.

In [ ]:
# Utilities

def sha1_str(s: str) -> str:
    return hashlib.sha1(s.encode('utf-8')).hexdigest()

def simple_clean(text: str) -> str:
    # Normalize newlines and spaces
    t = text.replace('\r\n', '\n').replace('\r', '\n')
    # Remove lines with only numbers (like page numbers)
    t = '\n'.join([line for line in t.split('\n') if not re.fullmatch(r'\s*\d+\s*', line)])
    # Collapse multiple newlines
    t = re.sub(r'\n{3,}', '\n\n', t)
    # Collapse multiple spaces
    t = re.sub(r'[ \t]{2,}', ' ', t)
    return t.strip()

def chunk_text_by_words(text: str, chunk_words: int = 300, overlap: int = 50):
    words = text.split()
    if len(words) <= chunk_words:
        return [' '.join(words)]
    chunks = []
    i = 0
    step = chunk_words - overlap
    while i < len(words):
        chunk = words[i:i+chunk_words]
        chunks.append(' '.join(chunk))
        i += step
    return chunks


In [ ]:
# Ingest Documents

def load_pdf_file(path):
    text = ""
    try:
        with open(path, "rb") as f:
            reader = PyPDF2.PdfReader(f)
            for page in reader.pages:
                text += page.extract_text() + "\n"
    except Exception as e:
        print(f"[WARN] Failed to read PDF {path}: {e}")
    return text

def ingest_documents_from_dir(input_dir):
    SUPPORTED_EXT = {".txt", ".md", ".csv", ".pdf"}
    docs = []
    for p in sorted(Path(input_dir).rglob("*")):
        if p.is_file() and p.suffix.lower() in SUPPORTED_EXT:
            try:
                if p.suffix.lower() == ".csv":
                    df = pd.read_csv(p)
                    text_col = next((c for c in ['text','content','body','summary'] if c in df.columns), None)
                    if text_col is None:
                        continue
                    for idx, row in df.iterrows():
                        txt = str(row[text_col])
                        docs.append({
                            'doc_id': sha1_str(str(p) + str(idx)),
                            'title': row.get('title', f"{p.name}-{idx}"),
                            'source': str(p),
                            'date': row.get('date', ''),
                            'text': txt
                        })
                elif p.suffix.lower() == ".pdf":
                    txt = load_pdf_file(p)
                    docs.append({
                        'doc_id': sha1_str(str(p)),
                        'title': p.stem,
                        'source': str(p),
                        'date': '',
                        'text': txt
                    })
                else:
                    txt = load_txt_file(p)
                    docs.append({
                        'doc_id': sha1_str(str(p)),
                        'title': p.stem,
                        'source': str(p),
                        'date': '',
                        'text': txt
                    })
            except Exception as e:
                print(f"Failed to read {p}: {e}")
    return docs

In [ ]:
# Preprocess & Chunk

def preprocess_and_chunk(docs, chunk_words=300, overlap=50):
    chunks = []
    for d in docs:
        cleaned = simple_clean(d.get('text',''))
        text_chunks = chunk_text_by_words(cleaned, chunk_words, overlap)
        for i, c in enumerate(text_chunks):
            chunk_id = sha1_str(d['doc_id'] + str(i))
            chunks.append({
                'doc_id': d['doc_id'],
                'chunk_id': chunk_id,
                'title': d.get('title',''),
                'source': d.get('source',''),
                'date': d.get('date',''),
                'chunk_text': c,
                'chunk_word_count': len(c.split()),
                'created_at': datetime.utcnow().isoformat()+'Z'
            })
    return chunks

## **4. Embedding and FAISS Indexing**
Convert text chunks to embeddings, store in FAISS, and save the index.

In [ ]:
# Embeddings

class EmbeddingEngine:
    def __init__(self, model_name="sentence-transformers/all-MiniLM-L6-v2"):
        self.model = SentenceTransformer(model_name)
    def embed(self, texts):
        return self.model.encode(texts, show_progress_bar=True, convert_to_numpy=True).astype('float32')

In [ ]:
# Build FAISS Index

def build_faiss_index(embeddings):
    if not HAS_FAISS:
        print("FAISS not installed. Skipping index creation.")
        return None
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)
    return index

In [ ]:
# Run Phase 1

def run_phase1_colab(input_dir, chunk_words=300, overlap=50):
    print("[INFO] Ingesting documents...")
    docs = ingest_documents_from_dir(input_dir)
    print(f"[INFO] Found {len(docs)} documents.")

    print("[INFO] Preprocessing and chunking...")
    chunks = preprocess_and_chunk(docs, chunk_words, overlap)
    print(f"[INFO] Created {len(chunks)} chunks.")

    print("[INFO] Computing embeddings...")
    texts = [c['chunk_text'] for c in chunks]
    emb_engine = EmbeddingEngine()
    embeddings = emb_engine.embed(texts)
    print(f"[INFO] Embeddings shape: {embeddings.shape}")

    print("[INFO] Building FAISS index...")
    index = build_faiss_index(embeddings)
    if index:
        print(f"[INFO] FAISS index built with {index.ntotal} vectors.")

    # Save outputs to Google Colab local runtime (optional)
    os.makedirs('processed_data', exist_ok=True)
    pd.DataFrame(chunks).to_csv('processed_data/chunks.csv', index=False)
    np.save('processed_data/embeddings.npy', embeddings)
    with open('processed_data/metadata.json','w') as f:
        json.dump(chunks, f, indent=2)
    print("[INFO] Phase 1 complete. Outputs saved in processed_data/ folder.")

In [ ]:
run_phase1_colab('data_docs')

[INFO] Ingesting documents...
[WARN] Failed to read PDF data_docs/wound_surgical.pdf: PyCryptodome is required for AES algorithm
[INFO] Found 7 documents.
[INFO] Preprocessing and chunking...
[INFO] Created 176 chunks.
[INFO] Computing embeddings...


/tmp/ipython-input-2318952334.py:19: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  'created_at': datetime.utcnow().isoformat()+'Z'
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

[INFO] Embeddings shape: (176, 384)
[INFO] Building FAISS index...
[INFO] FAISS index built with 176 vectors.
[INFO] Phase 1 complete. Outputs saved in processed_data/ folder.


## **5. Retrieval-Augmented Generation (RAG) Pipeline**
Retrieve top-k context chunks and generate responses with LLM.

In [ ]:
# Load stored embeddings + metadata
print("[INFO] Loading stored embeddings and metadata...")

chunks_df = pd.read_csv('processed_data/chunks.csv')
embeddings = np.load('processed_data/embeddings.npy')

with open('processed_data/metadata.json', 'r') as f:
    metadata = json.load(f)

print(f"[INFO] Loaded {len(chunks_df)} chunks with embeddings of shape {embeddings.shape}")

In [ ]:
# Load embedding model

EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

print(f"[INFO] Loading embedding model: {EMBEDDING_MODEL}")
embedding_model = SentenceTransformer(EMBEDDING_MODEL)

[INFO] Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


In [ ]:
embedding_model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [ ]:
# Build FAISS index
dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(embeddings)
print(f"[INFO] FAISS index built with {index.ntotal} vectors of dimension {dim}")

[INFO] FAISS index built with 176 vectors of dimension 384


In [ ]:
# Define retrieval function
def retrieve_top_docs(query, top_k=3, metadata_filter=None):
    """
    Retrieves top_k relevant chunks for a user query.
    Optionally applies metadata filtering.
    """
    # Embed the user query
    query_emb = embedding_model.encode([query])

    # Perform similarity search
    D, I = index.search(query_emb, top_k)

    results = []
    for idx, score in zip(I[0], D[0]):
        if idx >= len(chunks_df):
            continue
        chunk_info = chunks_df.iloc[idx].to_dict()

        if metadata_filter:
            valid = all(chunk_info.get(k) == v for k, v in metadata_filter.items())
            if not valid:
                continue

        results.append({
            "content": chunk_info["chunk_text"],
            "title": chunk_info.get("title", ""),
            "source": chunk_info.get("source", ""),
            "similarity_score": float(score)
        })
    return results

In [ ]:
# Test Retrieval Pipeline

sample_query = "How should I clean my surgical incision after knee surgery?"
results = retrieve_top_docs(sample_query, top_k=3)

print("\n[INFO] 🔍 Retrieval Results:\n")
for i, res in enumerate(results, start=1):
    print(f"Result {i}:")
    print(f"Title: {res['title']}")
    print(f"Source: {res['source']}")
    print(f"Similarity Score: {res['similarity_score']:.4f}")
    print(f"Chunk:\n{res['content'][:400]}...\n")


[INFO] 🔍 Retrieval Results:

Result 1:
Title: dr-volk-discharge-instructions-total-knee
Source: data_docs/dr-volk-discharge-instructions-total-knee.pdf
Similarity Score: 0.7143
Chunk:
you may begin to shower. The outer dressing may come off and the tape and glue underneath should b e left on. You can gently clean the incision with soap and warm water after 1 week. Gently pat dry and apply a new 4x4 gauze or adherent as needed. - Over the course of 2 -4 weeks the underneath tape and glue will come off. - Do not take tub baths, get in hot t ubs, or swimming pools, or soak your in...

Result 2:
Title: Recovery-After-Your-Knee-Booklet-VF
Source: data_docs/Recovery-After-Your-Knee-Booklet-VF.pdf
Similarity Score: 0.7573
Chunk:
your feet up, down, and in circles 10 times every hour when awake. 3. Buttock Contractions – Tighten your buttocks muscles and hold for a count of 5 seconds 3 to 5 times every day. CARING FOR YOUR INCISION It is important that you take proper care of your incision to

In [ ]:
# ================================================
# 📘Context Augmentation & LLM Generation
# ================================================

In [ ]:
# Load your open-source LLM
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

print("[INFO] 🔄 Loading LLM model and tokenizer...")
llm_tokenizer = AutoTokenizer.from_pretrained(model_name)
llm_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16,
    device_map="auto"
)
print("[INFO] ✅ Model Loaded Successfully!")


[INFO] 🔄 Loading LLM model and tokenizer...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[INFO] ✅ Model Loaded Successfully!


In [ ]:
llm_pipe = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    max_new_tokens=300,
    temperature=0.6,
    top_p=0.9,
    device_map="auto"
)

Device set to use cuda:0


In [ ]:
# 🧩 Prompt Template

def build_prompt(user_query, retrieved_docs, ehr_info=None):
    context_text = "\n\n".join(
        [f"Source: {doc['source']}\nContent: {doc['content']}" for doc in retrieved_docs]
    )

    ehr_context = ""
    if ehr_info:
        ehr_context = (
            f"Surgery Type: {ehr_info.get('surgery_type', 'N/A')}\n"
            f"Allergies: {ehr_info.get('allergies', 'N/A')}\n"
            f"Recovery Timeline: {ehr_info.get('recovery_timeline', 'N/A')}\n"
        )

    prompt = f"""
You are a post-operative care assistant.
Use the context to provide an accurate and safe medical response.

Patient Information:
{ehr_context}

Patient Query: {user_query}

Context Information:
{context_text}

Provide a concise, safe, and medically appropriate response:
"""
    return prompt.strip()

In [ ]:
# 🧠 Response Generation

def generate_response(prompt, max_new_tokens=300, temperature=0.7):
    """
    Generate only the new model response (not the full prompt).
    """
    inputs = llm_tokenizer(prompt, return_tensors="pt").to(llm_model.device)
    output_tokens = llm_model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        top_p=0.9,
        pad_token_id=llm_tokenizer.eos_token_id
    )
    full_text = llm_tokenizer.decode(output_tokens[0], skip_special_tokens=True)

    response_only = full_text[len(prompt):].strip()

    return response_only

In [ ]:
# 🧩 Connect Retrieval + Generation

def answer_patient_query(user_query, ehr_info=None, top_k=3):
    """
    Retrieves docs + patient info, builds prompt, and returns clean model output.
    """
    retrieved_docs = retrieve_top_docs(user_query, top_k=top_k)
    full_prompt = build_prompt(user_query, retrieved_docs, ehr_info)

    # Generate concise answer
    response_text = generate_response(full_prompt)
    return response_text

In [ ]:
# -------------------------------
# Example test
# -------------------------------
ehr_example = {
    "surgery_type": "Total Knee Replacement",
    "allergies": "Penicillin",
    "recovery_timeline": "2 weeks post-op"
}

user_query = "How should I clean my surgical incision after knee surgery?"
response = answer_patient_query(user_query, ehr_info=ehr_example, top_k=3)

print("\n[INFO] 🩺 Final AI Response:\n")
print(response)


[INFO] 🩺 Final AI Response:

You can gently clean your surgical incision with soap and warm water after one week post-op. Be sure to pat it dry and apply a new sterile dressing as needed. Avoid soaking your incision, including taking tub baths or swimming, until it is completely healed. Use ice therapy as directed by your healthcare provider to help reduce inflammation, swelling, and pain. Remember to elevate your operated leg above the level of your heart for 30-60 minutes at a time, 4-5 times a day to help reduce swelling. Always wash your hands before touching your incision, and report any signs of infection, such as yellow or green drainage and odor, to your healthcare provider.


## **6. Response Evaluation and Safety Filtering**
Categorize responses and flag unsafe/incomplete ones.

In [ ]:
# 🔹 Output Delivery & Source Linking

def display_patient_response(final_text):
    """
    Pretty print markdown formatted output.
    """
    display(Markdown(final_text))

# Example visualization
display_patient_response(response)

# Categorization logic
def categorize_response(response_text):
    if "🚨" in response_text:
        return "🚨 ER Visit"
    elif "⚠️" in response_text:
        return "⚠️ Provider Connect"
    else:
        return "✅ AI Recommendation"

category = categorize_response(response)
print(f"\n[INFO] Recommendation Category: {category}")


You can gently clean your surgical incision with soap and warm water after one week post-op. Be sure to pat it dry and apply a new sterile dressing as needed. Avoid soaking your incision, including taking tub baths or swimming, until it is completely healed. Use ice therapy as directed by your healthcare provider to help reduce inflammation, swelling, and pain. Remember to elevate your operated leg above the level of your heart for 30-60 minutes at a time, 4-5 times a day to help reduce swelling. Always wash your hands before touching your incision, and report any signs of infection, such as yellow or green drainage and odor, to your healthcare provider.


[INFO] Recommendation Category: ✅ AI Recommendation


In [ ]:
# 📊 Response Evaluation & Logging

def evaluate_response(user_query, response, retrieved_docs):
    """
    Performs a simple heuristic evaluation of the model output
    for post-operative medical guidance.
    Returns a dictionary with quality scores and flags.
    """

    eval_result = {
        "query": user_query,
        "response_length": len(response.split()),
        "has_warning_signs": any(keyword in response.lower() for keyword in ["infection", "bleeding", "emergency", "immediately"]),
        "contains_context_terms": any(doc["source"].split('/')[-1].replace('.pdf', '').lower() in response.lower() for doc in retrieved_docs),
        "is_concise": len(response.split()) < 200,
        "is_safe_tone": not any(term in response.lower() for term in ["ignore", "disregard", "self-medicate", "skip doctor"]),
        "timestamp": datetime.now().isoformat()
    }

    # Heuristic summary
    eval_result["summary"] = "✅ Safe & relevant" if eval_result["is_safe_tone"] and eval_result["is_concise"] else "⚠️ Needs review"

    return eval_result


def log_interaction(user_query, ehr_info, response, eval_result, save_path="interaction_logs.json"):
    """
    Logs query, patient info, response, and evaluation results to a JSON file.
    """

    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "query": user_query,
        "ehr_info": ehr_info,
        "response": response,
        "evaluation": eval_result
    }

    # If log file exists, append new entry; otherwise create new
    if os.path.exists(save_path):
        with open(save_path, "r", encoding="utf-8") as f:
            data = json.load(f)
    else:
        data = []

    data.append(log_entry)

    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

    print(f"[LOGGED] Interaction saved to {save_path}")

In [ ]:
# -------------------------------
#  Example Test
# -------------------------------
user_query = "How should I clean my surgical incision after knee surgery?"

response = answer_patient_query(user_query, ehr_info=ehr_example, top_k=3)

# Evaluate response quality
eval_result = evaluate_response(user_query, response, retrieve_top_docs(user_query))

# Log everything
log_interaction(user_query, ehr_example, response, eval_result)

print("\n[INFO] ✅ Evaluation Summary:")
print(json.dumps(eval_result, indent=4))

[LOGGED] Interaction saved to interaction_logs.json

[INFO] ✅ Evaluation Summary:
{
    "query": "How should I clean my surgical incision after knee surgery?",
    "response_length": 137,
    "has_warning_signs": false,
    "contains_context_terms": false,
    "is_concise": true,
    "is_safe_tone": true,
    "timestamp": "2025-10-15T15:18:49.846864",
    "summary": "\u2705 Safe & relevant"
}


In [ ]:
# 🛡️ Automated Safety Guardrail

def apply_safety_guardrail(response):
    """
    Enhanced contextual guardrail that checks for unsafe or misleading advice.
    Allows safe educational phrasing (e.g., 'do not skip antibiotics').
    """
    text = response.lower()

    # Absolute unsafe triggers (always blocked)
    unsafe_terms = [
        "self-medicate", "double dose", "stop taking without",
        "ignore your doctor", "use unprescribed", "apply heat directly"
    ]

    # Contextually risky terms (flag only if not negated)
    caution_terms = ["skip", "avoid", "delay"]

    # Detect strict unsafe content
    flagged_terms = [term for term in unsafe_terms if term in text]

    # Context-aware detection for 'skip'
    for term in caution_terms:
        # Example: "skip antibiotics" (unsafe), but not "do not skip antibiotics"
        unsafe_pattern = rf"\b((?!do not|don’t|never).)\b{term}\b.*(medicine|dose|antibiotic|exercise|therapy)"
        if re.search(unsafe_pattern, text):
            flagged_terms.append(term)

    if flagged_terms:
        print(f"⚠️ [GUARDRAIL] Potentially unsafe or misleading phrasing detected: {flagged_terms}")
        safe_response = (
            "⚠️ This response may include ambiguous phrasing. "
            "Please follow only your doctor's instructions or seek clarification from your healthcare provider."
        )
        return safe_response, True
    else:
        return response, False


# -------------------------------
# Example Integration
# -------------------------------
user_query = "Can I skip my antibiotics if I feel better after surgery?"
ehr_info = {
    "surgery_type": "Appendectomy",
    "allergies": "None",
    "recovery_timeline": "3 days post-op"
}

# Generate model answer
model_response = answer_patient_query(user_query, ehr_info=ehr_info)

# Apply safety guardrail
safe_response, flagged = apply_safety_guardrail(model_response)

# Evaluate and log
eval_result = evaluate_response(user_query, safe_response, retrieve_top_docs(user_query))
log_interaction(user_query, ehr_info, safe_response, eval_result)

print("\n[INFO] 🩺 Final Safe Response:")
print(safe_response)
print(f"\n[FLAGGED]: {flagged}")


[LOGGED] Interaction saved to interaction_logs.json

[INFO] 🩺 Final Safe Response:
It is important to complete the full course of antibiotics prescribed by your healthcare provider after an appendectomy to reduce the risk of post-surgical infections. Skipping antibiotics, even if you feel better, could increase the risk of complications. If you have any concerns or questions about your medication, please consult your healthcare provider.

[FLAGGED]: False


## **7. Gradio Interface**
Display clean UI for user inputs and AI-generated outputs.

In [ ]:

def gradio_postop_assistant(user_query, surgery_type, allergies, recovery_timeline, top_k=3):
    """
    Main function to connect user input to retrieval + LLM + safety guardrail
    """
    ehr_info = {
        "surgery_type": surgery_type,
        "allergies": allergies,
        "recovery_timeline": recovery_timeline
    }

    # 1️⃣ Generate model response
    model_response = answer_patient_query(user_query, ehr_info=ehr_info, top_k=top_k)

    # 2️⃣ Apply safety guardrail
    safe_response, flagged = apply_safety_guardrail(model_response)

    # 3️⃣ Evaluate response
    eval_result = evaluate_response(user_query, safe_response, retrieve_top_docs(user_query, top_k=top_k))

    # 4️⃣ Log interaction
    log_interaction(user_query, ehr_info, safe_response, eval_result)

    # 5️⃣ Categorize
    category = categorize_response(safe_response)

    # 6️⃣ Prepare output
    output_text = f"**Response Category:** {category}\n\n**Response:**\n{safe_response}"

    return output_text, flagged, json.dumps(eval_result, indent=2)


# Gradio UI Components

with gr.Blocks() as demo:
    gr.Markdown("## 🩺 Post-Operative Care Assistant")
    gr.Markdown(
        "Enter your patient query and relevant surgical/EHR information. "
        "The assistant will retrieve context, generate a safe response, "
        "and categorize it."
    )

    with gr.Row():
        with gr.Column():
            user_query = gr.Textbox(label="Patient Query", placeholder="Describe your question...")
            surgery_type = gr.Textbox(label="Surgery Type", placeholder="e.g., Total Knee Replacement")
            allergies = gr.Textbox(label="Allergies", placeholder="e.g., Penicillin")
            recovery_timeline = gr.Textbox(label="Recovery Timeline", placeholder="e.g., 2 weeks post-op")
            top_k = gr.Slider(1, 10, value=3, step=1, label="Number of Context Chunks to Retrieve")
            submit_btn = gr.Button("Get Advice")

        with gr.Column():
            response_output = gr.Markdown(label="Assistant Response")
            flagged_output = gr.Textbox(label="⚠️ Flagged?", interactive=False)
            eval_output = gr.Textbox(label="Evaluation Details", lines=10, max_lines=20, interactive=False, placeholder="Evaluation summary will appear here...")

    submit_btn.click(
        fn=gradio_postop_assistant,
        inputs=[user_query, surgery_type, allergies, recovery_timeline, top_k],
        outputs=[response_output, flagged_output, eval_output]
    )

## 8. Run App

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f7c07b22480dffddea.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
